In [1]:
from ultralytics import YOLO
import torch

In [2]:
class EarlyStopping:
    def __init__(self, patience=5, delta=0):
        """
        Args:
        - patience (int): The number of epochs with no improvement after which training will be stopped.
        - delta (float): Minimum change to qualify as an improvement.
        """
        self.patience = patience
        self.delta = delta
        self.best_loss = None
        self.patience_counter = 0

    def should_stop(self, current_loss):
        """Check whether to stop training."""
        if self.best_loss is None:
            self.best_loss = current_loss
            return False
        elif current_loss < self.best_loss - self.delta:
            self.best_loss = current_loss
            self.patience_counter = 0
        else:
            self.patience_counter += 1
        
        if self.patience_counter >= self.patience:
            print(f"Early stopping triggered at epoch {epoch}")
            return True
        
        return False

def main():
    # Check if GPU is available
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Using device: {device}")

    # Load the pretrained YOLOv8 medium classification model
    model = YOLO('yolov8m-cls.pt').to(device)

    # Initialize early stopping
    early_stopping = EarlyStopping(patience=5, delta=0.001)

    # Train the model
    for epoch in range(100):  # Train for 100 epochs
        print(f"Epoch {epoch+1}/100")

        # Train for one epoch
        results = model.train(
            data='datasets\diff-types',  # path to your dataset (root folder)
            epochs=1,                   # train for one epoch at a time
            batch=8,                    # batch size
            imgsz=224,                  # image size
            device=0,                   # use GPU (0 means first GPU)
            workers=4,                  # number of worker threads
            optimizer='Adam',           # optimizer
            lr0=0.001,                  # initial learning rate
            seed=42                     # random seed
        )

        # Get the current validation loss (this will depend on your model)
        current_loss = results.metrics['val/loss']  # assuming validation loss is available
        
        # Check if early stopping should be triggered
        if early_stopping.should_stop(current_loss):
            print("Stopping early due to no improvement.")
            break

    # Save the trained model
    model.save('e-waste-classifier.pt')

In [3]:
if __name__ == '__main__':
    main()

Using device: cuda
Epoch 1/100
Ultralytics 8.3.109  Python-3.11.3 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 3050 Ti Laptop GPU, 4096MiB)
engine\trainer: task=classify, mode=train, model=yolov8m-cls.pt, data=datasets\diff-types, epochs=1, time=None, patience=100, batch=8, imgsz=224, save=True, save_period=-1, cache=False, device=0, workers=4, project=None, name=train, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=42, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_label

100%|██████████| 5.35M/5.35M [00:00<00:00, 14.5MB/s]


AMP: checks passed 


train: Scanning R:\Projects\E-Waste Management\Codes\datasets\diff-types\train... 2400 images, 0 corrupt: 100%|██████████| 2400/2400 [00:00<?, ?it/s]
val: Scanning R:\Projects\E-Waste Management\Codes\datasets\diff-types\val... 300 images, 0 corrupt: 100%|██████████| 300/300 [00:00<?, ?it/s]


optimizer: Adam(lr=0.001, momentum=0.937) with parameter groups 38 weight(decay=0.0), 39 weight(decay=0.0005), 39 bias(decay=0.0)
Image sizes 224 train, 224 val
Using 4 dataloader workers
Logging results to runs\classify\train
Starting training for 1 epochs...

      Epoch    GPU_mem       loss  Instances       Size


               classes   top1_acc   top5_acc: 100%|██████████| 19/19 [00:00<00:00, 19.45it/s]


                   all       0.92      0.997

1 epochs completed in 0.009 hours.
Optimizer stripped from runs\classify\train\weights\last.pt, 31.7MB
Optimizer stripped from runs\classify\train\weights\best.pt, 31.7MB

Validating runs\classify\train\weights\best.pt...
Ultralytics 8.3.109  Python-3.11.3 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 3050 Ti Laptop GPU, 4096MiB)
YOLOv8m-cls summary (fused): 42 layers, 15,775,466 parameters, 0 gradients, 41.6 GFLOPs
train: R:\Projects\E-Waste Management\Codes\datasets\diff-types\train... found 2400 images in 10 classes  
val: R:\Projects\E-Waste Management\Codes\datasets\diff-types\val... found 300 images in 10 classes  
test: R:\Projects\E-Waste Management\Codes\datasets\diff-types\test... found 300 images in 10 classes  


               classes   top1_acc   top5_acc: 100%|██████████| 19/19 [00:00<00:00, 28.27it/s]


                   all       0.92      0.997
Speed: 0.2ms preprocess, 1.8ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs\classify\train


AttributeError: 'ClassifyMetrics' object has no attribute 'metrics'. See valid attributes below.

    Class for computing classification metrics including top-1 and top-5 accuracy.

    Attributes:
        top1 (float): The top-1 accuracy.
        top5 (float): The top-5 accuracy.
        speed (dict): A dictionary containing the time taken for each step in the pipeline.
        task (str): The task type, set to 'classify'.
    